In [1]:
from term_grouping import *
from print_cliques import print_cliques
import matplotlib.pyplot as plt
from collections import Counter
import time
import numpy as np
import glob
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, get_interaction_operator
from openfermionpsi4 import run_psi4
from openfermion.transforms import (jordan_wigner, bravyi_kitaev, bravyi_kitaev_fast, bravyi_kitaev_tree,
                                    binary_code_transform, reverse_jordan_wigner)
from openfermion.utils import group_into_tensor_product_basis_sets
from openfermion.transforms._binary_codes import parity_code

plt.rcParams["font.size"] = 13
plt.rcParams["axes.labelsize"] = 15
plt.rcParams["axes.titlesize"] = 15
plt.rcParams["font.family"] = 'sans-serif'
plt.rcParams["xtick.labelsize"] = 13
plt.rcParams["ytick.labelsize"] = 13

In [2]:
def load_transform_group(filename, encoding_AS_dict):
    # Load molecule from file
    print('--- loading molecule ---')
    molecule = MolecularData(filename=filename)
    print('filename: {}'.format(molecule.filename))
    print('--- compute integrals ---')
    molecule = run_psi4(molecule,run_mp2=True,run_cisd=True,run_ccsd=True,run_fci=True)
    
    #print(molecule.two_body_integrals)
    #print(molecule.canonical_orbitals)
    #print('n_atoms: {}'.format(molecule.n_atoms))
    #print('n_electrons: {}'.format(molecule.n_electrons))
    #print('n_orbitals: {}'.format(molecule.n_orbitals))
    #print('Canonical Orbitals: {}'.format(molecule.canonical_orbitals))
    #print('n_qubits: {}'.format(molecule.n_qubits))
    
    # construct Hamiltonian for each pair of transform-AS in encoding_AS_dict
    data = []
    for transform in encoding_AS_dict.keys():
        print('\n>>>> Using {} transform'.format(transform))
        all_orbital_pairs = encoding_AS_dict[transform]
        for orbital_pair in all_orbital_pairs:
            
            occupied_num, active_num = orbital_pair
            print('ActiveSpace = {}'.format(active_num))
            
            # Construct Hamiltonian
            molecular_hamiltonian = molecule.get_molecular_hamiltonian(
                                    occupied_indices=range(occupied_num),
                                    active_indices=range(active_num))
    
            # map the operator to fermions and then qubits
            fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
    
            # get interaction operator (needed for BKSF)
            interaction_hamiltonian = get_interaction_operator(fermion_hamiltonian)
    
            if transform is 'JW':
                qubit_h = jordan_wigner(fermion_hamiltonian)
                qubit_h.compress()
            elif transform is 'BK':
                qubit_h = bravyi_kitaev(fermion_hamiltonian)
                qubit_h.compress()
            elif transform is 'BKSF':
                qubit_h = bravyi_kitaev_fast(interaction_hamiltonian)
                qubit_h.compress()
            elif transform is 'BKT':
                qubit_h = bravyi_kitaev_tree(fermion_hamiltonian)
                qubit_h.compress()
            elif transform is 'PC':
                qubit_h = binary_code_transform(fermion_hamiltonian, parity_code(2*active_num))
                qubit_h.compress()
            else:
                print('ERROR: Unrecognized qubit transformation: {}'.format(transform))
                sys.exit(2)
                
            num_terms = len(qubit_h.terms.keys()) - 1
            print('OPENFERMION: generated Hamiltonian with {} terms (excluding Identity)'.format(num_terms))
                
            # Use OpenFermion function to group into commuting sets using QWC
            # time the computation
            start_time = time.time()
            cliques = group_into_tensor_product_basis_sets(qubit_h)
            end_time = time.time()
            
            print('OPENFERMION: group_into_tensor_product_basis_sets found {} unique circuits'.format(len(cliques)))
            et = end_time - start_time
            print('OPENFERMION: Elapsed time: {:.6f}s'.format(et))
            
            data.append((num_terms,len(cliques),et))
            
    return data

In [49]:
ch4filename = 'molecule_data/CH4_sto-3g_singlet_grnd.hdf5'
ch4dict = {'BK': [(9,1),(9,2),(9,3),(9,4),(9,5),(9,6),(9,7),(9,8),(9,9)]}

h2filename1 = 'molecule_data/H2_6-31g_singlet_0.7.hdf5'
h2dict1 = {'BK': [(4,1),(4,2),(4,3),(4,4)],
           'BKSF': [(4,4)], 'BKT': [(4,4)],
           'JW': [(4,1),(4,2),(4,3),(4,4)],
           'PC': [(4,4)]}

h2filename2 = 'molecule_data/H2_sto-3g_singlet_0.7.hdf5'
h2dict2 = {'BK': [(2,1),(2,2)]}

h2ofilename1 = 'molecule_data/H2O_6-31g_singlet_104.hdf5'
h2odict1 = {'BK': [(6,1),(6,2),(6,3),(6,4),(6,5),(6,6)],
           'BKSF': [(6,6)], 'BKT': [(6,6)],
           'JW': [(6,6)],
           'PC': [(6,6)]}

h2ofilename2 = 'molecule_data/H2O_sto-3g_singlet_104.hdf5'
h2odict2 = {'BK': [(6,1),(6,2),(6,3),(6,4),(6,5),(6,6)]}

lihfilename = 'molecule_data/LiH_sto-3g_singlet_1.45.hdf5'
lihdict = {'BK': [(6,1),(6,2),(6,3),(6,4),(6,5),(6,6)]}

filelist = [ch4filename, h2filename1, h2filename2, h2ofilename1, h2ofilename2, lihfilename]
dictlist = [ch4dict,     h2dict1,     h2dict2,     h2odict1,     h2odict2,     lihdict]

all_data = []
for fname, encode_dict in zip(filelist,dictlist):
    cur_data = load_transform_group(fname, encode_dict)
    all_data.append(cur_data)

--- loading molecule ---
filename: molecule_data/CH4_sto-3g_singlet_grnd
--- compute integrals ---

>>>> Using BK transform
ActiveSpace = 1
OPENFERMION: generated Hamiltonian with 3 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 1 unique circuits
OPENFERMION: Elapsed time: 0.001992s
ActiveSpace = 2
OPENFERMION: generated Hamiltonian with 26 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 8 unique circuits
OPENFERMION: Elapsed time: 0.002163s
ActiveSpace = 3
OPENFERMION: generated Hamiltonian with 93 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 33 unique circuits
OPENFERMION: Elapsed time: 0.003411s
ActiveSpace = 4
OPENFERMION: generated Hamiltonian with 240 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 70 unique circuits
OPENFERMION: Elapsed time: 0.010487s
ActiveSpace = 5
OPENFERMION: generated Hamiltonian with 591 terms (excluding Identity)


>>>> Using BK transform
ActiveSpace = 1
OPENFERMION: generated Hamiltonian with 3 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 1 unique circuits
OPENFERMION: Elapsed time: 0.002077s
ActiveSpace = 2
OPENFERMION: generated Hamiltonian with 26 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 8 unique circuits
OPENFERMION: Elapsed time: 0.002313s
ActiveSpace = 3
OPENFERMION: generated Hamiltonian with 117 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 40 unique circuits
OPENFERMION: Elapsed time: 0.004573s
ActiveSpace = 4
OPENFERMION: generated Hamiltonian with 192 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 55 unique circuits
OPENFERMION: Elapsed time: 0.007850s
ActiveSpace = 5
OPENFERMION: generated Hamiltonian with 275 terms (excluding Identity)
OPENFERMION: group_into_tensor_product_basis_sets found 65 unique circuits
OPENFERMION: Elapsed t

In [50]:
# write results to file
with open('Data/openfermion_clique_cover_results.txt', 'w') as fn:
    for dset in all_data:
        for run in dset:
            nterms, ncliques, runtime = run
            fn.write('{0} {1} {2:.6f}\n'.format(str(nterms).ljust(5), str(ncliques).ljust(5), runtime))